<a href="https://colab.research.google.com/github/CarlosLopes88/04_Optimization_models/blob/main/Prob_max_lucro_mix_prod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema

Seja uma empresa que produz quatro produtos, A, B, C e D. A fabricação de cada uma unidade desses produtos exige mão-de-obra, matéria-prima e processamento mecânico, gerando um lucro, de acordo com a tabela abaixo:

| Recursos / Produtos | A | B | C | D | Disponibilidade |
|:--- |:---:|:---:|:---:|:---:|:---:|
| <b>Mão-de-obra(homens-hora/unidade)</b> | 8 | 3 | 5 | 6 | 15.000 hh |
| <b>Matéria-prima(kg/unidade)</b> | 5 | 7 | 4 | 5 | 20.000 kg |
| <b>Processamento mecânico(horas-máquina)</b> | 12 | 9 | 8 | 7 | 40.000 hm |
| <b>Lucro($/unidade)</b> | 3 | 6 | 5 | 4 | |

O suprimento semanal de matéria-prima é restrito a 20.000 kg. A disponibilidade semanal de mão-de-obra é de 15.000 horas, e a quantidade de horas-máquina é de 40.000 hm. Pede-se determinar o plano de produção semanal, de forma a maximizar o lucro.

# Modelagem

### Variáveis de decisão:

$x_1 \colon \text{quantidade A produzido}$

$x_2 \colon \text{quantidade B produzido}$

$x_3 \colon \text{quantidade C produzido}$

$x_4 \colon \text{quantidade D produzido}$

### Função objetivo:

$\mathrm{max} \; Z = 3*x_1 + 6*x_2 + 5*x_3 + 4*x_4$

### Restrições:

$8*x_1 + 3*x_2 + 5*x_3 + 6*x_4 \leq 15000$

$5*x_1 + 7*x_2 + 4*x_3 + 5*x_4 \leq 20000$

$12*x_1 + 9*x_2 + 8*x_3 + 7*x_4 \leq 40000$

$x_1, x_2, x_3, x_4 \geq 0$

In [ ]:
# Instalando Gurobi e Pyomo

!pip install gurobipy
!pip install pyomo

In [ ]:
# Importando as bibliotécas

import pyomo.environ as pyo
import gurobipy as gp
from pyomo.core.base.set import IntegerSet

In [ ]:
# Dados de entrada:

lucros = [3, 6, 5, 4]

parâmetros = [[ 8,	3,	5,	6], 
             [5,	7,	4,	5	], 
             [12,	9,	8,	7	]]

limites = [15000, 20000, 40000]

In [ ]:
m = len(limites)

n = len(lucros)

In [ ]:
# Declaração do modelo:

modelo = pyo.ConcreteModel()

In [ ]:
# Declaração dos conjuntos:

modelo.I = pyo.RangeSet(m)
modelo.J = pyo.RangeSet(n)

In [ ]:
# Declaração dos parâmetros:

modelo.c = pyo.Param(modelo.J, initialize=lambda modelo, j: lucros[j-1])

In [ ]:
modelo.a = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: parâmetros[i-1][j-1])

In [ ]:
modelo.b = pyo.Param(modelo.I, initialize=lambda modelo, i: limites[i-1])

In [ ]:
# Declaração das variáveis de decisão:

modelo.x = pyo.Var(modelo.J, within=pyo.NonNegativeIntegers)

In [ ]:
# Declaração da função objetivo:

def regra_fo(mod):
    return pyo.summation(mod.c, mod.x)

In [ ]:
modelo.z = pyo.Objective(rule=regra_fo, sense=pyo.maximize)

In [ ]:
# Declaração das restrições:

def regra_restricao(mod, i):
    return sum(mod.a[i,j] * mod.x[j] for j in mod.J) <= mod.b[i]

In [ ]:
modelo.disponibilidade_mo = pyo.Constraint(modelo.I, rule=regra_restricao)

In [ ]:
modelo.pprint()

1 Set Declarations
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :   12 : {(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (3, 1), (3, 2), (3, 3), (3, 4)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=4, Bounds=(1, 4)
        Key  : Finite : Members
        None :   True :   [1:4]

3 Param Declarations
    a : Size=12, Index=a_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :     8
        (1, 2) :     3
        (1, 3) :     5
        (1, 4) :     6
        (2, 1) :     5
        (2, 2) :     7
        (2, 3) :     4
        (2, 4) :     5
        (3, 1) :    12
        (3, 2) :     9
        (3, 3) :     8
        (3, 4) :     7
    b : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 : 15000
          2 : 20000
  

In [ ]:
# Aplicando o solver

resultado = pyo.SolverFactory("gurobi", solver_io="python").solve(modelo)

In [ ]:
resultado.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 20215.0
  Upper bound: 20217.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 4
  Number of binary variables: 0
  Number of integer variables: 4
  Number of continuous variables: 0
  Number of nonzeros: 12
  Sense: -1
  Number of solutions: 3
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 9.50
  Status: ok
  Wallclock time: 0.0007021427154541016
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solu

In [ ]:
modelo.x.pprint()

x : Size=4, Index=J
    Key : Lower : Value  : Upper : Fixed : Stale : Domain
      1 :     0 :   -0.0 :  None : False : False : NonNegativeIntegers
      2 :     0 : 1740.0 :  None : False : False : NonNegativeIntegers
      3 :     0 : 1955.0 :  None : False : False : NonNegativeIntegers
      4 :     0 :   -0.0 :  None : False : False : NonNegativeIntegers


In [ ]:
modelo.z()

20215.0